In [2]:
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()
os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("openai_api_key")

In [4]:
# !pip install langchain
!pip install langchain_community
# !pip install openai
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 8.1 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [67]:
import warnings
warnings.filterwarnings("ignore", module="langchain")

In [6]:
# from langchain.llms import OpenAI
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.7)

In [76]:
def printf(data, col=4):
    col_width = max(len(str(x)) for x in data)
    for i, x in enumerate(data, 1):
        print(f"{x:<{col_width}}", end="\n" if i % col == 0 else "")

def printt(tuples):
    for tuple_ in tuples:
        print(tuple_[0])
        print(tuple_[1])
        print()

In [53]:
# Print all the attributes of llm object
printt([(i, getattr(llm, i)) for i in dir(llm) if not i.startswith("_")])

InputType
typing.Union[str, langchain_core.prompt_values.StringPromptValue, langchain_core.prompt_values.ChatPromptValueConcrete, list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='Sy

In [50]:
printf(dir(llm))

InputType                     OutputType                    __abstractmethods__           __annotations__               
__call__                      __class__                     __class_getitem__             __class_vars__                
__copy__                      __deepcopy__                  __delattr__                   __dict__                      
__dir__                       __doc__                       __eq__                        __fields__                    
__fields_set__                __format__                    __ge__                        __get_pydantic_core_schema__  
__get_pydantic_json_schema__  __getattr__                   __getattribute__              __getstate__                  
__gt__                        __hash__                      __init__                      __init_subclass__             
__iter__                      __le__                        __lt__                        __module__                    
__ne__                        __

In [51]:
llm.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'cache': FieldInfo(annotation=Union[BaseCache, bool, NoneType], required=False, default=None, exclude=True),
 'verbose': FieldInfo(annotation=bool, required=False, default_factory=_get_verbosity, exclude=True, repr=False),
 'callbacks': FieldInfo(annotation=Union[list[BaseCallbackHandler], BaseCallbackManager, NoneType], required=False, default=None, exclude=True),
 'tags': FieldInfo(annotation=Union[list[str], NoneType], required=False, default=None, exclude=True),
 'metadata': FieldInfo(annotation=Union[dict[str, Any], NoneType], required=False, default=None, exclude=True),
 'custom_get_token_ids': FieldInfo(annotation=Union[Callable[list, list[int]], NoneType], required=False, default=None, exclude=True),
 'callback_manager': FieldInfo(annotation=Union[BaseCallbackManager, NoneType], required=False, default=None, exclude=True),
 'client': FieldInfo(annotation=Any, required=False, default=None, exclud

# Langchain

In [54]:
print(llm("What is langchain?"))



Langchain is a term that is often used in the field of linguistics to refer to a network of interconnected languages. This can include languages that are genetically related, as well as those that have influenced each other through contact and borrowing. The term is also sometimes used to describe the similarities and differences between languages and how they are related to each other. Langchain can also refer to the study of these language networks. 


In [61]:
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.7)

name_template = PromptTemplate(
    input_variables = ['cuisine'],
    template = 'I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.'
)

name_template.format(cuisine="bread")

'I want to open a restaurant for bread food. Suggest a fancy name for this.'

- This is like a f string, so why don't we just use that?

In [65]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=name_template)
chain.run("bread")

'\n\n"Bread & Beyond"'

## Sequential Chain

In [38]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

name_template = PromptTemplate(
    input_variables = ['cuisine'],
    template = 'I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.'
)

name_chain = LLMChain(
    llm = OpenAI(temperature=0.7),
    prompt = name_template,
    output_key = 'restaurant_name'
)


item_template = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items from {restaurant_name}. Only a list of item names."
)

item_chain = LLMChain(
    llm = OpenAI(temperature=0.8),
    prompt = item_template,
    output_key = 'menu_items'
)

In [39]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, item_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

chain({'cuisine':'bread'})

{'cuisine': 'bread',
 'restaurant_name': '\n\n"Crust & Co." ',
 'menu_items': '\n1. Margherita Pizza\n2. Pepperoni Pizza\n3. Mushroom and Truffle Oil Pizza\n4. Classic Caesar Salad\n5. Caprese Salad\n6. Chicken Pesto Panini\n7. Meatball Sub\n8. Tuscan Chicken Alfredo\n9. Baked Ziti\n10. Beef Lasagna\n11. Spinach and Feta Calzone\n12. Garlic Knots\n13. Antipasto Platter\n14. Prosciutto and Arugula Flatbread\n15. Grilled Vegetable Panini\n16. Roasted Garlic and Parmesan Breadsticks\n17. Chicken Parmesan Sandwich\n18. Shrimp Scampi Linguine\n19. Eggplant Parmesan\n20. Tiramisu Cheesecake.'}

## Making an App using Streamlit

- powerful library for building proof of concept apps

```python
import streamlit as st
from langchain_helper import generate

st.title("Restaurant Name Generator")

cuisine = st.sidebar.selectbox("Pick a cuisine", ("Italian", "Chinese", "French", "Spanish", "Greek", "Indian", "Japanese", "Korean", "Mexican", "Thai"))

# def generate(cusine):
#     return chain({'cuisine': cusine})

if cuisine:
    response = generate(cuisine)

    st.header(response['restaurant_name'].replace('"', ''))
    menu_items = response['menu_items'].split(',')
    st.write('**Menu Items:**')
    for item in menu_items:
        st.write(item.strip())
```

- run this locally

In [9]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=6a295c9762fd423ba53b33511d1e91b132b4ecebdd68a1f23b2a2e38134d0de8
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


## Langchain Agents

In [14]:
from langchain.agents import AgentType, initialize_agent, load_tools

llm = OpenAI(temperature=0.7)

tools = load_tools(
    ['wikipedia', 'llm-math'],
    llm = llm,
)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True # can be used in most langchain functions to list steps
)

In [15]:
agent.run('When and where was bread first made?')



> Entering new AgentExecutor chain...
 I should use wikipedia to search for the history of bread
Action: wikipedia
Action Input: "history of bread"
Observation: Page: History of bread
Summary: Bread was central to the formation of early human societies. From the Fertile Crescent, where wheat was domesticated, cultivation spread north and west, to Europe and North Africa, and east toward East Asia. This in turn led to the formation of towns, which curtailed nomadic lifestyles, and gave rise to other forms of societal organization. Similar developments occurred in the Americas with maize and in Asia with rice.



Page: Bread
Summary: Bread is a staple food prepared from a dough of flour (usually wheat) and water, usually by baking. Throughout recorded history and around the world, it has been an important part of many cultures' diet. It is one of the oldest human-made foods, having been of significance since the dawn of agriculture, and plays an essential role in both religious rituals

'Bread was first made in the Fertile Crescent, specifically in the region of Mesopotamia, around 8000 BC.'

In [16]:
agent.run('''What is the population of the united states in 2024?
How much would it cost to give each person in the US 1 loaf of bread, if that costs $3.61''')



> Entering new AgentExecutor chain...
 We can use Wikipedia to find the population of the United States in 2024.
Action: Wikipedia
Action Input: "population of united states 2024"
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: We can use the Calculator tool to find the total cost.
Action: Calculator
Action Input: 331,002,651 * 3.61
Observation: Answer: 1194919570.11
Thought: I now know the final answer.
Final Answer: It would cost $1,194,919,570.11 to give each person in the US 1 loaf of bread.

> Finished chain.


'It would cost $1,194,919,570.11 to give each person in the US 1 loaf of bread.'

## DuckDuckGo Search Agent

- free API

In [19]:
!pip install -qU duckduckgo-search langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.6 MB/s eta 0:00:0000:0100:01


In [20]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Obama's last name?")

'Barack Obama is opening up about Malia Obama\'s decision to drop their family name in her professional life — and how his daughters feel about remaining in the spotlight as they grow older Malia Obama is voting for a fresh start.. As her Hollywood career takes off, Barack and Michelle Obama\'s oldest daughter has dropped her last name in favor of her middle name, officially going by ... During an appearance on "The Pivot Podcast," former President Barack Obama revealed that his now 26-year-old eldest daughter used a different surname in the credits of her short film "The ... Former President Barack Obama recently opened up about his eldest daughter\'s decision to drop her famous last name in a professional setting. Keen to carve her own path and not rest on the high-powered Obama name, Malia has chosen to go by Malia Ann, her first name and middle name, for her newest film project, The Heart. "Fans did a ...'

In [28]:
tools = load_tools(
    ['wikipedia', 'llm-math'],
    llm=llm,
)

tools.append(search)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [32]:
agent.run("Who won the Nobel Peace Prize in 2024?")



> Entering new AgentExecutor chain...
 This is a question about current events, so I should use duckduckgo_search to find the answer.
Action: duckduckgo_search
Action Input: "Nobel Peace Prize 2024"
Observation: The Norwegian Nobel Committee has decided to award the Nobel Peace Prize for 2024 to the Japanese organisation Nihon Hidankyo. This grassroots movement of atomic bomb survivors from Hiroshima and Nagasaki, also known as Hibakusha, is receiving the Peace Prize for its efforts to achieve a world free of nuclear weapons and for demonstrating ... The 2024 Nobel Peace Prize has been awarded to Nihon Hidankyo, a grassroots Japanese organization of atomic bomb survivors from Hiroshima and Nagasaki for its efforts "to achieve a world free of ... The Japanese grassroots group Nihon Hidankyo, an organization of World War II atomic bombing survivors, has won the 2024 Nobel Peace Prize for its efforts to create a world free of nuclear weapons. Nihon Hidankyo, a Japanese group of atomic b

'The winner of the Nobel Peace Prize in 2024 was Nihon Hidankyo, a Japanese organization of atomic bomb survivors.'

## Memory

In [43]:
chain.memory  # This is None

In [64]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

name_template = PromptTemplate(
    input_variables = ['cuisine'],
    template = 'I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.'
)

name_chain = LLMChain(
    llm = OpenAI(temperature=0.7),
    prompt = name_template,
    output_key = 'restaurant_name',
    memory = memory,
)

name_chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]))

In [65]:
result = name_chain({'cuisine':'bread'})
name_chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='bread', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Bread & Beyond: A Gourmet Bread Experience"', additional_kwargs={}, response_metadata={})]))

## Memory Buffer Window
- This memory buffer keeps growing
- So restrict the buffer size

In [69]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [70]:
convo.run("Who is the youngest chess grandmaster?")

' The youngest chess grandmaster is Abhimanyu Mishra, who achieved the title at the age of 12 years, 4 months, and 25 days. He is from the United States and became a grandmaster on July 1, 2021.\n'

In [72]:
print(convo.memory)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Who is the youngest chess grandmaster?', additional_kwargs={}, response_metadata={}), AIMessage(content=' The youngest chess grandmaster is Abhimanyu Mishra, who achieved the title at the age of 12 years, 4 months, and 25 days. He is from the United States and became a grandmaster on July 1, 2021.\n', additional_kwargs={}, response_metadata={})])


In [73]:
convo.run("How old are they, as of 2024?")

' As of 2024, Abhimanyu Mishra will be 15 years old.'

In [82]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=llm,
    memory = memory
)

convo.run("Who is the youngest chess grandmaster?")

' The youngest chess grandmaster is Abhimanyu Mishra, who achieved the title at the age of 12 years, 4 months, and 25 days. He is from the United States and was officially awarded the title by FIDE (World Chess Federation) in July 2021. He broke the record of Sergey Karjakin, who became a grandmaster at the age of 12 years and 7 months in 2002. Is there anything else you would like to know?'

In [83]:
convo.run("What is 2+2?")

'  2+2 is 4. This is a basic mathematical equation that is taught in elementary school. It is also known as addition, where you add two numbers together to get a sum. Is there anything else I can help you with?'

In [84]:
convo.run("How old are they, as of 2024?")

' I do not have enough information to accurately answer that question. I am an AI and do not have access to personal data such as age. Is there something else you would like to know?'